In [1]:
import gym
import json
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from gym import wrappers




In [2]:
from env.StockTradingEnv import StockTradingEnv
from networks.DQNac import MyModel
from networks.DQNac import StockActor, StockCritic

In [3]:
df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

# The algorithms require a vectorized environment to run
env = StockTradingEnv(df)

base_action = np.array([0,0.5])

obs = env.reset()
for i in range(5):
    action, _states = (base_action, None)
    obs, rewards, done, info = env.step(action)
    
obs=np.expand_dims(obs,axis=0)
obs=np.expand_dims(obs,axis=0)
#print(obs)


In [4]:
act = StockActor()
act_target = StockActor()
output_action = act.predict(obs)
output_action_tar = act_target.predict(obs)
print("output_action")
print(output_action)
print(" ")
print("output_action_tar")
print(output_action_tar)
print(" ")
output_action = act.predict(obs)
output_action_tar = act_target.predict(obs)
print("output_action2")
print(output_action)
print(" ")
print("output_action_tar2")
print(output_action_tar)
print(" ")

output_action
tf.Tensor([[-0.00185421]], shape=(1, 1), dtype=float32)
 
output_action_tar
tf.Tensor([[-0.00012242]], shape=(1, 1), dtype=float32)
 
output_action2
tf.Tensor([[-0.00185421]], shape=(1, 1), dtype=float32)
 
output_action_tar2
tf.Tensor([[-0.00012242]], shape=(1, 1), dtype=float32)
 


In [5]:
model = MyModel(32, (1,4), (1,1))
moutput=tf.squeeze(model(obs))
a=tf.squeeze(moutput)
print(a[0])
print(output_action[0][0])
b = [a[0], a[1], output_action[0][0]]
b = tf.convert_to_tensor(b)
print(b)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(-0.025225773, shape=(), dtype=float32)
tf.Tensor(-0.0018542098, shape=(), dtype=float32)
tf.Tensor([-0.02522577 -0.05698976 -0.00185421], shape=(3,), dtype=float32)


In [6]:

#model = MyModel(32, (1,4), (1,1))
#output=tf.squeeze(model(obs))

crit = StockCritic()
crit_tar = StockCritic()

output = crit.predict(obs,output_action)
output_tar = crit_tar.predict(obs,output_action)
print("output")
print(output)
print(" ")
print("output_tar")
print(output_tar)
print(" ")
output = crit.predict(obs,output_action)
output_tar = crit_tar.predict(obs,output_action)
print("output2")
print(output)
print(" ")
print("output_tar2")
print(output_tar)
print(" ")


ValueError: in converted code:

    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\P3A-Deep-Reinforcement-Learning\Stock-Trading-Environment\networks\DQNac.py:101 build_critic  *
        z = tf.convert_to_tensor(a)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\framework\ops.py:1256 convert_to_tensor_v2
        as_ref=False)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\framework\ops.py:1314 convert_to_tensor
        ret = conversion_func(value, dtype=dtype, name=name, as_ref=as_ref)
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\ops\array_ops.py:1368 _autopacking_conversion_function
        return _autopacking_helper(v, dtype, name or "packed")
    C:\Users\Thomas\PycharmProjects\reinforcement-learning-master\venv\lib\site-packages\tensorflow_core\python\ops\array_ops.py:1304 _